# Loading Saved Model Weights (Deployment)

<p style="font-weight:bold">DS-413 | 2023 First Semester</p>

Instructor: Jhun Brian M. Andam \
Department of Data Science \
College of Information Technology and Computing \
University of Science and Technology of Southern Philippines

- https://www.gradio.app/guides/building-a-pictionary-app

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

torch.backends.cudnn.deterministic=True
torch.set_printoptions(sci_mode=False)

import cv2
import time

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, (3,3), 1, 1)
        self.pool1 = nn.MaxPool2d((2,2), 2)
        self.conv2 = nn.Conv2d(32, 64, (3,3), 1, 1)
        self.pool2 = nn.MaxPool2d((2,2), 2)
        self.fcn1 = nn.Linear(64*7*7, 128)
        self.fcn2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 64*7*7)
        x = self.fcn1(x)
        x = F.softmax(self.fcn2(x), dim=1)
        
        return x

In [6]:
model = CNN()
model

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (fcn1): Linear(in_features=3136, out_features=128, bias=True)
  (fcn2): Linear(in_features=128, out_features=10, bias=True)
)

In [7]:
model.load_state_dict(torch.load('model/best_CNN.pth'))

<All keys matched successfully>

In [74]:
LABELS = list(range(0,10))
def predict(img):
    if img is None:
        return {}
        
    x = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.
    with torch.no_grad():
        out = model(x)
    values, indices = torch.topk(out, 10)
    values = values.view(-1)
    indices = indices.view(-1)
    confidences = {LABELS[i]: v.item() for i, v in zip(indices, values)}
    return confidences

In [75]:
import gradio as gr

gr.Interface(fn=predict,
             inputs="sketchpad",
             outputs="label",
             live=True).launch()

Running on local URL:  http://127.0.0.1:7862

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.
